This notebook takes an array of coldkeys and finds the emission on every hotkey associated with the coldkey.
This works for miners/validators and also stakeholders.

1. ADd your API key in the next cell (get one at dash.taostats.io)
2. choose your start and end dates in the format YYYY-MM-DD
3. Create an array of coldkeys that you wish to track

What happens:

first cell - stup the urls
2nd cell - silly date fixer - sometimes the chain adds MS, someties it does not :man shrugging:
3rd cell - query the API to get the daily close price for tao for the date range
4th: initialise the data
5th: Get the dailye data for each coldkey. There is a slepp to prevent any API rate limiting (after a few price calls that might happen)
6th: print one result
7th: math the daiy change by comparing day over day balance changes.
8th: get any registrations and caluclate those
9 & 10 - create a JSON and CSV for the created data

coldkey, hotkey, date, income, expense, price

your CK, the HK associated, day, tao earned, tao spent, tao price in $

the hotkey can be your miner, your validator.. if you are staking it is vthe hotkey of the validator you staked with.


In [28]:
import requests, json
import pandas as pd
import numpy as np
import decimal
import time
from datetime import datetime, timedelta
from rich import print


api_key=""
start_date= "2024-01-01"
end_date="2025-01-01"
networks = ["finney", "nakamoto", "kusanagi"]
coldkeys =[
""
  
]

# USD, GBP, EUR, etc
currency = "USD"

headers = {
            "accept": "application/json",
            "Authorization": api_key
        }


In [29]:
#sometimes the chain spits out a time with milliseconds.. Sometimess it doesnt.  Its annoying.
#this fixes it. returns a datetime object
def stupid_time_fix(timestamp):
    if len(timestamp) > 20:
        date_obj = datetime.strptime(timestamp, "%Y-%m-%dT%H:%M:%S.%fZ")
    else:
        date_obj = datetime.strptime(timestamp, "%Y-%m-%dT%H:%M:%SZ")
    
    return date_obj

In [30]:
#get the close price of tao for every date in the range

date_obj = datetime.strptime(start_date, "%Y-%m-%d")
unix_start = int(date_obj.timestamp())-87000*3
date_obj = datetime.strptime(end_date, "%Y-%m-%d")
unix_end = int(date_obj.timestamp())

print(unix_start, unix_end)

#get all the tao prices
all_price_data = []

next_page=1

while next_page is not None:
    price_url=f"https://api.taostats.io/api/price/ohlc/v1?asset=tao&period=1d&timestamp_start={unix_start}&timestamp_end={unix_end}&limit=200&page={next_page}"
    response = requests.get(price_url, headers=headers)
    #print(response.text)
    resJson = json.loads(response.text)
    next_page = resJson['pagination']['next_page']
    all_price_data += resJson['data']

print(len(all_price_data))
date_price = {}
for price in all_price_data:
    pricetime  = stupid_time_fix(price['timestamp'])
    date = pricetime.date()
    date_str = date.strftime("%Y-%m-%d")
    close = price['close']
    date_price[date_str]=close

1703824200 1735707600

369

In [31]:
#print(resJson)
income_totals = {}
daily_stake = {}
daily_stake_date = {}
stake_balances = []
neuron_registrations = []

In [32]:

for coldkey in coldkeys:
    for network in networks:
        pages = 1
        page =1
    
        url = f"https://api.taostats.io/api/accounting/v1?network={network}&date_start={start_date}&date_end={end_date}&coldkey={coldkey}&page={page}"
        
        response = requests.get(url, headers=headers)
        #print(response.text)
        resJson = json.loads(response.text)
        income = float(resJson['data'][0]['income'])/1e9
        reg_costs = float(resJson['data'][0]['neuron_registration_cost'])/1e9
        pages = resJson['pagination']['total_pages']
        print(network, coldkey, income, reg_costs)
        stake_balances += resJson['data'][0]['stake_balances']
        neuron_registrations  += resJson['data'][0]['neuron_registrations']
        time.sleep(1)

finney 5HayDwsVBpGDExka66oTpc5bWQKYK57atEJej9skQ56gGP8F 260.578693058 0.0

nakamoto 5HayDwsVBpGDExka66oTpc5bWQKYK57atEJej9skQ56gGP8F 0.0 0.0

kusanagi 5HayDwsVBpGDExka66oTpc5bWQKYK57atEJej9skQ56gGP8F 0.0 0.0

finney 5G6vvxdMJWrVk2ChUZgAuBegKVCsy3uAzegMohCWNVqwhPso 22.876254339 0.0

nakamoto 5G6vvxdMJWrVk2ChUZgAuBegKVCsy3uAzegMohCWNVqwhPso 0.0 0.0

...

nakamoto 5F7PwP2ysRFzhBoSV3qrKwvJYJm8V8dCvXJUAAePJjX1GUjy 0.0 0.0

kusanagi 5F7PwP2ysRFzhBoSV3qrKwvJYJm8V8dCvXJUAAePJjX1GUjy 0.0 0.0

finney 5GzwqFeaJgg2sxzMmYU9jVzgijxx9xeDSgp7gU8PA6VH7r7r 0.0 0.0

nakamoto 5GzwqFeaJgg2sxzMmYU9jVzgijxx9xeDSgp7gU8PA6VH7r7r 0.0 0.0

kusanagi 5GzwqFeaJgg2sxzMmYU9jVzgijxx9xeDSgp7gU8PA6VH7r7r 0.0 0.0

In [33]:
total =0
income_totals = {}
daily_stake = {}
daily_stake_date = {}

initial_stake = float(stake_balances[0]['stake'])/1e9
for index, balance in enumerate(stake_balances):
    if index >=0:
        newColdkey = False
        timestamp = stupid_time_fix(balance['timestamp'])
        date = timestamp.date()
        yesterday = date-timedelta(days=1)
        date_str = date.strftime("%Y-%m-%d")
        yesterday_date_str  = yesterday.strftime("%Y-%m-%d")
        hk = balance['hotkey']['ss58']
        ck= balance['coldkey']['ss58']
        stake_balance = float(balance['stake'])/1e9
        added = float(balance['added'])/1e9
        removed  = float(balance['removed'])/1e9
        if ck not in income_totals:
            income_totals[ck] = {}
            newColdkey=True
        if hk not in income_totals[ck]:
            income_totals[ck][hk] = {}
        if date_str not in income_totals[ck][hk]:
            #print(date_str)
            income_totals[ck][hk][date_str] = {"stake": 0, "added":0, "removed":0,"total":0, "dod":0, "price": date_price[date_str]}
        #get yesterdays stake, if it exists
        yesterday_stake = 0
        if yesterday_date_str in income_totals[ck][hk]:
            yesterday_stake = income_totals[ck][hk][yesterday_date_str]['stake']
        #print(date_str, yesterday_date_str,yesterday_stake, newColdkey)
        #stake is a static value. we want the last value for the day.
        #if we add it each time, the last value of teh day will be the final value
        income_totals[ck][hk][date_str]['stake'] = stake_balance

        # added  must be a sum of all stake added to the hotkey in this day
        income_totals[ck][hk][date_str]['added'] += added
        #removed is a sum of all stake removved during the day
        income_totals[ck][hk][date_str]['removed'] += removed

        if newColdkey:
            #total earned is  - added + removed
            #for a new ck we ignore the instake - it is not income it was there before hhis loop started
            
            income_totals[ck][hk][date_str]['total'] = - income_totals[ck][hk][date_str]['added'] +income_totals[ck][hk][date_str]['removed']
    
            
        else:
            #total earned is currnet stake - added + removed
            income_totals[ck][hk][date_str]['total'] = income_totals[ck][hk][date_str]['stake'] - income_totals[ck][hk][date_str]['added'] +income_totals[ck][hk][date_str]['removed']
    
        #dod is today's total minus yesterdays stake
        income_totals[ck][hk][date_str]['dod'] = income_totals[ck][hk][date_str]['total'] - yesterday_stake
            
#print(income_totals)


# So daily_stake_date has the total staked value for the day for all 

In [34]:

for reg in neuron_registrations:
    timestamp = stupid_time_fix(reg['timestamp'])
    date = timestamp.date()
    date_str = date.strftime("%Y-%m-%d")
    reg_cost = float(reg['registration_cost'])/1e9
    hk = reg['hotkey']['ss58']
    ck = reg['coldkey']['ss58']
    if ck not in income_totals:
            income_totals[ck] = {}
    if hk not in income_totals[ck]:
            income_totals[ck][hk] = {}
    if date_str in income_totals[ck][hk]:
        if 'expense' in income_totals[ck][hk][date_str]:
            income_totals[ck][hk][date_str]['expense'] += round(reg_cost,3)
        else:
            #new entry
            income_totals[ck][hk][date_str]['expense'] = round(reg_cost,3)
            income_totals[ck][hk][date_str]['price'] =  date_price[date_str]
            income_totals[ck][hk][date_str]['coldkey'] = ck
    else:
        #new data and enw expense
        income_totals[ck][hk][date_str] = {}
        income_totals[ck][hk][date_str]['expense'] = round(reg_cost,3)
        income_totals[ck][hk][date_str]['price'] =  date_price[date_str]
        income_totals[ck][hk][date_str]['coldkey'] = ck


In [35]:
# Open a text file and write the JSON object to it
with open('incometotals.json', 'w') as f:
    json.dump(income_totals, f)
    

In [36]:
totals_csv = "coldkey , hotkey , date , income , expense , price \n"
for coldkey, hotkeyValues in  income_totals.items():
    for hotkey, dateValues in hotkeyValues.items():
        for date, prices in dateValues.items():
            income = prices.get('dod',0)
            expense = prices.get('expense',0)
            price = prices.get('price',0)
            totals_csv += f"{coldkey} , {hotkey}, {date}, {income}, {expense}, {price} \n"
        
      
       
        

In [37]:
import csv
import io

with open('incometotalsv2.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    reader = csv.reader(io.StringIO(totals_csv), delimiter=',', quotechar='"')
    for row in reader:
        writer.writerow(row)